<a href="https://colab.research.google.com/github/FrancescoCarrabino/depthOBJ/blob/main/ModelTL_depthOBJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MODEL Notebook**

---


In this notebook, we train a model from the [Inception ResNet V2](http://ai.googleblog.com/2016/08/improving-inception-and-image.html) architecture on the depth maps contained in the dataset DepthOBJ.

The model we produce will then be imported in the Visualizer Notebook, to categorize depth maps and retrieve a category-representative 3D mesh in .OBJ format using [PyTorch3D](https://pytorch3d.org/)


<img src="https://drive.google.com/uc?id=1rI1vtTp4oYYZS-a6SZ5nQaeP4ewOT4Xe" width="500">

> *You can visit the dataset repository on* [GitHub](https://github.com/francesco-carrabino/depthOBJ).






## ***Imports for data and preprocessing***


In [ ]:
import os
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from matplotlib.image import imread
from tensorflow.keras.utils import plot_model

## ***Load the data from Google Drive***

In [ ]:
# mount drive [only on colab]
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Datasets'

In [ ]:
!unzip depthOBJ.zip -d '/content'

In [ ]:
# directories of DepthOBJ
data_dir = '/content/content/drive/MyDrive/Datasets/depthOBJ/depthOBJ'
images_path = data_dir + '/Images/'
models_path = data_dir + '/Models/'

## ***ImageDataGenerator***

---



The Data Generator integreated in Keras, called [ImageDataGenerator](https://keras.io/api/preprocessing/image/), allows the user to load and augment the data in real-time, while saving computational power by processing the data in batches. In fact, after setting a reasonable batch size, the ImageDataGenerator loads the images in the dataset and applies randomly to them the real-time modifications that the user set, for example vertical or horizontal shift, different angles, rotations and so on. %The Neural Networks, therefore, never actually sees the original image and never sees the same image twice. 



<img src='https://pyimagesearch.com/wp-content/uploads/2019/07/keras_data_augmentation_header.png'>


In [ ]:
#import the data generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# create the data generator to handle images from folders
datagen = ImageDataGenerator(rotation_range=5, 
                             horizontal_flip=True, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1,
                             validation_split = 0.2)

In [ ]:
# set a variable for resizing images
image_shape = (299, 299, 3)

## ***Inception ResNet V2***

---



Making a case for sparsely-connected Neural Network, Google presented in 2010 Inception V1, introducing the Inception layer, from which the architecture takes the name. This layer is a combination of three different Convolutional layers 1x1, 3x3 and 5x5, in which the output is the concatenation of each Convolutional layer's output. Therefore, the Inception layer allows the network to use filters of the right dimension every time.

Inception ResNet V2 is an architecture which combines the theory behind Inception and ResNet, using three different Inception layers and shortcuts for *skip-connections* and reaching 50.6 million parameters.

<img src='https://2.bp.blogspot.com/-9KD48z54MBs/V8cVz11fM0I/AAAAAAAABKM/sCC0vVEz_dMOsyb0D8AFwqkrrCavdlkSACLcB/s1600/image02.png'>

### Import the model from Keras

In [ ]:
#import models and layers from tf.keras 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Activation, Add, Input, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import load_model

### Instantiate a model from Inception ResNet V2
 
We instantiate the model with pre-trained weights. The weights have been trained on the images of [ImageNet](http://www.image-net.org/). We use the Transfer-Learning technique to re-train the weights on our dataset adding two last Dense output layers. 

<img src='https://qjjnh3a9hpo1nukrg1fwoh71-wpengine.netdna-ssl.com/wp-content/uploads/2019/12/cover_transfer_learning_1600px_web-1024x512.jpg'>

In [ ]:
# initiate a model from Inception ResNet V2 architecture and compile it
model = InceptionResNetV2(weights='imagenet', 
                          input_shape=image_shape, 
                          classes=54,
                          pooling='max',
                          include_top=False)
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(54, activation='softmax')(class1)

model = Model(model.inputs, output)

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(learning_rate=0.0002), 
              metrics=['accuracy'])
model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1015 (Conv2D)            (None, 149, 149, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1015 (Batch (None, 149, 149, 32) 96          conv2d_1015[0][0]                
__________________________________________________________________________________________________
activation_1015 (Activation)    (None, 149, 149, 32) 0           batch_normalization_1015[0][0]   
_______________________________________________________________________________________

In [ ]:
plot_model(model)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.586886 to fit



## ***Training the model***


---

Instantiated and compiled the model, we then proceeded to train it on the augmented data generated by the ImageDataGenerator. Model will been trained for 30 epochs, using a batch-size of 16.



> The training could take up to 8 hours.



### Generate the training and the validation data using the flow_from_directory function of the ImageDataGenerator

In [ ]:
# set a batch size to feed to the model
batch_size = 16

# generate data for the training
train_image_gen = datagen.flow_from_directory(images_path,
                                              target_size=image_shape[:2], 
                                              color_mode='rgb', 
                                              batch_size=batch_size, 
                                              class_mode='categorical', 
                                              shuffle=True,
                                              subset = 'training')

# generate data for the validation
valid_image_gen = datagen.flow_from_directory(images_path,
                                              target_size=image_shape[:2],
                                              color_mode='rgb', 
                                              batch_size=batch_size, 
                                              class_mode='categorical', 
                                              shuffle=False,
                                              subset = 'validation')

Found 15552 images belonging to 54 classes.
Found 3888 images belonging to 54 classes.


### Create callback function ModelCheckpoint for saving the best model during the training

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('/content/drive/MyDrive/Trained Models/inception_resnet_v2_TL.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
callback_list = [mc]

###  Set the steps in each epoch for the training

In [ ]:
TRAIN_STEPS = train_image_gen.n//batch_size
print(TRAIN_STEPS)
VALID_STEPS = valid_image_gen.n//batch_size
print(VALID_STEPS)

972
243


### **Train the model**

In [ ]:
losses = model.fit(x=train_image_gen, 
                   epochs=30, 
                   steps_per_epoch=TRAIN_STEPS, 
                   validation_data=valid_image_gen, 
                   validation_steps=VALID_STEPS, 
                   callbacks=callback_list)


Epoch 1/30
972/972 [==============================] - ETA: 0s - loss: 1.8517 - accuracy: 0.5006
Epoch 00001: val_accuracy improved from -inf to 0.75026, saving model to /content/drive/MyDrive/Trained Models/inception_resnet_v2_TL.h5
972/972 [==============================] - 7690s 8s/step - loss: 1.8517 - accuracy: 0.5006 - val_loss: 0.8585 - val_accuracy: 0.7503
Epoch 2/30
972/972 [==============================] - ETA: 0s - loss: 0.7871 - accuracy: 0.7637
Epoch 00002: val_accuracy improved from 0.75026 to 0.84131, saving model to /content/drive/MyDrive/Trained Models/inception_resnet_v2_TL.h5
972/972 [==============================] - 666s 685ms/step - loss: 0.7871 - accuracy: 0.7637 - val_loss: 0.5077 - val_accuracy: 0.8413
Epoch 3/30
972/972 [==============================] - ETA: 0s - loss: 0.5363 - accuracy: 0.8342
Epoch 00003: val_accuracy improved from 0.84131 to 0.87346, saving model to /content/drive/MyDrive/Trained Models/inception_resnet_v2_TL.h5
972/972 [==================

In [ ]:
while True:pass

## ***Save the history of the training in a CSV file***

In [ ]:
# save the history of the training
losses = pd.DataFrame(model.history.history)

In [ ]:
# convert the history of the training to a CSV
losses.to_csv(r'/content/drive/MyDrive/Trained Models/inception_losses_TL.csv', index = False, header = True)

## ***Evaluate the model***

In [ ]:
final_model = load_model('/content/drive/MyDrive/Trained Models/inception_resnet_v2_TL.h5')

In [ ]:
final_model.evaluate(valid_image_gen, batch_size=batch_size, steps=VALID_STEPS)

243/243 [==============================] - 132s 543ms/step - loss: 0.1085 - accuracy: 0.9673


[0.10845808684825897, 0.9673354029655457]